# MarkItDown
Wanted to see if we could use MarkItDown to make the resume better fit expected input of LLMs.

In [ ]:
from markitdown._markitdown import PdfConverter

md = PdfConverter()
resume_markdown = md.convert("Tommy Nguyen Resume 2 Pages.pdf", file_extension = ".pdf")

markup_text = resume_markdown.text_content.replace("• ", "").replace("• \n", "")

print(markup_text)

In [ ]:
from pypdf import PdfReader

reader = PdfReader("Tommy Nguyen Resume 2 Pages.pdf")

print(len(reader.pages))

final_resume_text = ""

for page_num in range(len(reader.pages)):
    resulting_text = reader.pages[page_num].extract_text()

    # Removing all "comments" brought over from Word to PDF when converting. Forgot to remove them beforehand. Preprocessing the text here.
    resulting_list = resulting_text.split("\n")
    for i in range(len(resulting_list)):
        index_of_comments = resulting_list[i].lower().find("comment")
        if index_of_comments >= 0:
            resulting_list[i] = resulting_list[i].replace(resulting_list[i][index_of_comments:-1], "")

        string_check = resulting_list[i].lower().strip()
        if "graduated august 2023..." in string_check or "undergraduate courses."  == string_check or "https://www.indeed.com/career -advice/resumes"  in string_check or "how did you mentor them?"  in string_check or "letters/how -to-list-publications -on-resume" in string_check or "another subsection as personal." == string_check:
            resulting_list[i] = None
    
    # TODO: There's still a ton of whitespaces between some parts, most likely due to page breaks.
    processed_text = "\n".join([item for item in resulting_list if item is not None ])

    if final_resume_text != "":
        final_resume_text += "\n\n" + processed_text
    else:
        final_resume_text += processed_text
    # print(f"Page {page_num}:\n{processed_text}")

print(f"Final extracted text:\n{final_resume_text}")

# Scraping For Job Postings

In [ ]:
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
l=[]
o={}
k=[]
space_replacer = "%20"
comma_replacer = "%2C"

keyword = ["Software Engineer", "Amazon"]
location = "Boston, MA"

keyword_query = f"{comma_replacer}{space_replacer}".join(keyword).replace(",", comma_replacer).replace(" ", space_replacer)
location_query = location.replace(",", comma_replacer).replace(" ", space_replacer)

print(keyword_query)
print(location_query)

url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={keyword_query}&location={location_query}"
target_url= url + '&geoId=&currentJobId=&start={}'

request = requests.get(f"https://www.linkedin.com/jobs/search?keywords={keyword_query}&location={location_query}&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
soup1 = BeautifulSoup(request.text, "html.parser")
print(f"https://www.linkedin.com/jobs/search?keywords={keyword_query}&location={location_query}&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")

results_context_class = "results-context-header"
results_context_tag_type = "div"

job_count_class = "results-context-header__job-count"
job_count_tag_type = "span"
try:
    num_jobs_found = soup1.find_all(results_context_tag_type, {"class" : results_context_class})[0].find(job_count_tag_type, {"class" : job_count_class}).text
except:
    raise Exception("The 'Job Counter' tag was not found! LinkedIn must have changed it...")

if not num_jobs_found or not num_jobs_found.isdigit():
    raise Exception("The returned job counter is invalid!")

num_jobs = int(num_jobs_found)
for i in range(0,math.ceil(num_jobs/25)):

    res = requests.get(target_url.format(i))
    soup=BeautifulSoup(res.text,'html.parser')
    alljobs_on_this_page=soup.find_all("li")
    print(len(alljobs_on_this_page))
    for x in range(0,len(alljobs_on_this_page)):
        jobid = alljobs_on_this_page[x].find("div",{"class":"base-card"}).get('data-entity-urn').split(":")[3]
        l.append(jobid)

target_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
for j in range(0,len(l)):

    resp = requests.get(target_url.format(l[j]))
    soup=BeautifulSoup(resp.text,'html.parser')

    try:
        o["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        o["company"]=None

    try:
        o["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        o["job-title"]=None

    try:
        o["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        o["level"]=None



    k.append(o)
    o={}

df = pd.DataFrame(k)
df.to_csv('linkedinjobs.csv', index=False, encoding='utf-8')
print(df)